# Continuation from GNN_Hands_on_Part_1

## Previously we explored : 

1. Introduction to Graph Data Structure in PyG
2. Homogeneous Data Example
3. Loading a Default Dataset
4. Creating Custom Graph Data
5. Splitting the Data
6. Working with Mini-Batches

## Table of Contents for the Current Notebook

7. Creating a GNN Model
8. Defining Train and Evaluation Methods
9. Demonstrating Node Classification Task
10. Demonstrating Graph Classification Task

## 7. Creating a GNN Model

To work with graph data, we need to define a Graph Neural Network (GNN) model. PyG provides a variety of GNN layers that can be used to build models for node classification, graph classification, and other tasks.

We'll cover:
- Choosing the appropriate GNN layer (e.g., `GCNConv`, `GATConv`, etc.)
- Defining the GNN architecture (input, hidden, and output layers)
- Incorporating activation functions, dropout, and other regularization techniques

## 8. Defining Train and Evaluation Methods

Once we have our model, it's important to define methods for training and evaluating its performance. In this section, we'll build the core training loop and evaluation function.

We'll explore:
- Setting up a loss function (e.g., Cross Entropy for classification)
- Optimizing with an appropriate optimizer (e.g., Adam)
- Tracking performance metrics (e.g., accuracy, loss)
- How to implement the training loop with backpropagation
- How to evaluate the model on validation and test sets


## 9. Demonstrating Node Classification Task

Node classification is a common task in GNNs, where the goal is to predict labels for individual nodes in a graph.

We'll demonstrate:
- How to set up a node classification task using a default or custom dataset
- Training the GNN model to classify nodes
- Evaluating performance using accuracy or other relevant metrics
- Visualizing node predictions (optional)


## 10. Demonstrating Graph Classification Task

Graph classification is another important task, where the aim is to predict a label for an entire graph.

Let's understand :
- How to structure a graph classification task using a suitable dataset
- Modifying the GNN model for graph-level predictions
- Training and evaluating the model for graph classification
- Techniques for improving performance, such as pooling layers (e.g., `global_mean_pool`)